In [1]:
%env SPARK_VERSION=3.0.0 # TODO PUT YOUR VALUE

env: SPARK_VERSION=3.0.0 # TODO PUT YOUR VALUE


In [2]:
import pydeequ
from pyspark.sql import SparkSession, Row
from pydeequ.repository import *

spark = (SparkSession
    .builder
    .config("spark.driver.extraClassPath", "/home/jovyan/work/mssql-jdbc-9.4.0.jre11.jar")
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())

In [3]:
server_url = "jdbc:sqlserver://host.docker.internal"
database_name = "AdventureWorks2012"
username = "sa"
password = "sa"
table = "Production.Product"
url = server_url + ";" + "databaseName=" + database_name + ";"
driver_class = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

jdbcDF = spark.read \
    .format("jdbc") \
    .option("url", url) \
    .option("dbtable", table) \
    .option("user", username) \
    .option("password", password) \
    .option("TRUSTED_CONNECTION", 'NO') \
    .load()

# TODO connect to DB with Spark using JDBC connection to read the data

In [4]:
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(jdbcDF) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("DaysToManufacture")) \
                    .addAnalyzer(Mean("ListPrice")) \
                    .addAnalyzer(DataType("Color")) \
                    .addAnalyzer(Distinctness("ProductNumber")) \
                    .addAnalyzer(Maximum("Weight")) \
                    .addAnalyzer(MaxLength("Name")) \
                    .addAnalyzer(Sum("StandardCost")) \
                    .run()
                    
analysisResult = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult.show()

+-------+-----------------+--------------------+-------------------+
| entity|         instance|                name|              value|
+-------+-----------------+--------------------+-------------------+
| Column|           Weight|             Maximum|             1050.0|
| Column|    ProductNumber|        Distinctness|                1.0|
| Column|            Color|      Histogram.bins|                5.0|
| Column|            Color|Histogram.abs.Boo...|                0.0|
| Column|            Color|Histogram.ratio.B...|                0.0|
| Column|            Color|Histogram.abs.Fra...|                0.0|
| Column|            Color|Histogram.ratio.F...|                0.0|
| Column|            Color|Histogram.abs.Int...|                0.0|
| Column|            Color|Histogram.ratio.I...|                0.0|
| Column|            Color|Histogram.abs.Unk...|              248.0|
| Column|            Color|Histogram.ratio.U...|0.49206349206349204|
| Column|            Color|Histogr

/usr/local/spark/python/pyspark/sql/dataframe.py:148: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [5]:
### Data profiling section
# TODO profile data here
from pydeequ.profiles import *

result = ColumnProfilerRunner(spark) \
    .onData(jdbcDF) \
    .run()

for col, profile in result.profiles.items():
    print(profile)

Py4JJavaError: An error occurred while calling o74.run.
: java.lang.NoSuchMethodError: 'org.apache.spark.sql.catalyst.expressions.aggregate.AggregateExpression org.apache.spark.sql.catalyst.expressions.aggregate.AggregateFunction.toAggregateExpression(boolean)'
	at org.apache.spark.sql.DeequFunctions$.withAggregateFunction(DeequFunctions.scala:31)
	at org.apache.spark.sql.DeequFunctions$.stateful_approx_count_distinct(DeequFunctions.scala:60)
	at com.amazon.deequ.analyzers.ApproxCountDistinct.aggregationFunctions(ApproxCountDistinct.scala:52)
	at com.amazon.deequ.analyzers.runners.AnalysisRunner$.$anonfun$runScanningAnalyzers$3(AnalysisRunner.scala:319)
	at scala.collection.immutable.List.flatMap(List.scala:366)
	at com.amazon.deequ.analyzers.runners.AnalysisRunner$.liftedTree1$1(AnalysisRunner.scala:319)
	at com.amazon.deequ.analyzers.runners.AnalysisRunner$.runScanningAnalyzers(AnalysisRunner.scala:318)
	at com.amazon.deequ.analyzers.runners.AnalysisRunner$.doAnalysisRun(AnalysisRunn

In [6]:
from pydeequ.suggestions import *

suggestionResult = ConstraintSuggestionRunner(spark) \
    .onData(jdbcDF) \
    .addConstraintRule(RetainTypeRule()) \
    .run()

# Constraint Suggestions in JSON format
print(json.dumps(suggestionResult, indent=2))

Py4JJavaError: An error occurred while calling o76.run.
: java.lang.NoSuchMethodError: 'org.apache.spark.sql.catalyst.expressions.aggregate.AggregateExpression org.apache.spark.sql.catalyst.expressions.aggregate.AggregateFunction.toAggregateExpression(boolean)'
	at org.apache.spark.sql.DeequFunctions$.withAggregateFunction(DeequFunctions.scala:31)
	at org.apache.spark.sql.DeequFunctions$.stateful_approx_count_distinct(DeequFunctions.scala:60)
	at com.amazon.deequ.analyzers.ApproxCountDistinct.aggregationFunctions(ApproxCountDistinct.scala:52)
	at com.amazon.deequ.analyzers.runners.AnalysisRunner$.$anonfun$runScanningAnalyzers$3(AnalysisRunner.scala:319)
	at scala.collection.immutable.List.flatMap(List.scala:366)
	at com.amazon.deequ.analyzers.runners.AnalysisRunner$.liftedTree1$1(AnalysisRunner.scala:319)
	at com.amazon.deequ.analyzers.runners.AnalysisRunner$.runScanningAnalyzers(AnalysisRunner.scala:318)
	at com.amazon.deequ.analyzers.runners.AnalysisRunner$.doAnalysisRun(AnalysisRunn

In [7]:
### Constraint Verification section
# TODO check selected constraints here and make beautify the report
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Review Check")

checkResult = VerificationSuite(spark) \
        .onData(jdbcDF) \
        .addCheck(
        check.hasMin("SafetyStockLevel", lambda x: x == 3) \
        .isComplete("StandardCost") \
        .isUnique("ProductID") \
        .isContainedIn("Color", ["Black", "Silver", "Blue", "Red"]) \
        .isNonNegative("DaysToManufacture")) \
        .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

report_df = checkResult_df.toPandas()
report_df.to_csv("my_report.csv")


Python Callback server started!
+------------+-----------+------------+--------------------+-----------------+--------------------+
|       check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+------------+-----------+------------+--------------------+-----------------+--------------------+
|Review Check|    Warning|     Warning|MinimumConstraint...|          Success|                    |
|Review Check|    Warning|     Warning|CompletenessConst...|          Success|                    |
|Review Check|    Warning|     Warning|UniquenessConstra...|          Success|                    |
|Review Check|    Warning|     Warning|ComplianceConstra...|          Failure|Value: 0.88690476...|
|Review Check|    Warning|     Warning|ComplianceConstra...|          Success|                    |
+------------+-----------+------------+--------------------+-----------------+--------------------+

